In [2]:

import pandas as pd
import numpy as np
import pickle
import csv
import glob
import altair as alt

import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.float_format', lambda x: '%.3f' % x)



In [3]:
from sqlalchemy import URL, create_engine

hostname = 'database-3.cluster-cgldqbnitk50.us-east-1.rds.amazonaws.com'
username = 'postgres'
password = 'cas_analytics'
database = 'postgres'

url_object = URL.create(
    "postgresql",
    username=username,
    password=password,  # plain (unescaped) text
    host=hostname,
    database=database,
)

engine = create_engine(url_object)

In [4]:
#get tsy data
tsy_query = "select * from cas.tsy_hist"
tsy_df = pd.read_sql(tsy_query, engine)
tsy_df.index = pd.DatetimeIndex(tsy_df['DATE'])
tsy_df['DGS10'] = pd.to_numeric(tsy_df['DGS10'], errors='coerce')
me_tsy_df = tsy_df.groupby(pd.Grouper(freq="M")).max()
me_tsy_df['ME_DATE'] = pd.to_datetime(me_tsy_df['DATE']).dt.to_period('M').dt.to_timestamp()


In [5]:
cols = [
    'Monthly Reporting Period',
    'Reference Pool ID',
    'Property State',
    'Loan Identifier',
    'Original Interest Rate',
    'Current Interest Rate',
    'Original UPB',
    'Current Actual UPB',
    'Loan Age',
    'Original Loan to Value Ratio (LTV)',
    'Original Combined Loan to Value Ratio (CLTV)',
    'Borrower Credit Score At Issuance',
    'Borrower Credit Score Current ',
    'Debt-To-Income (DTI)',
    'UPB at the Time of Removal',
    'Repurchase Date',
    'Zero Balance Code',
    'Zero Balance Effective Date',
    'Zero Balance Code Change Date',
    'Scheduled Principal Current',
    'Total Principal Current',
    'Unscheduled Principal Current',
    'Current Loan Delinquency Status'
]

ref_pools = ['1474']

cols_query = '"' + '", "'.join(cols) + '" '
ref_pools_query = "('" + "', '".join(ref_pools) + "') "

query = \
"""select """ \
+ cols_query + \
"""from cas.ref
where 
    "Reference Pool ID" in """ + ref_pools_query + \
"""limit 100000000;
"""

df = pd.read_sql(query, engine)


In [6]:
#clean df

tmp = df.copy()
tmp['ASOF_DATE'] = "01" + np.where(tmp['Monthly Reporting Period'].astype(str).str.len() == 5, 
                         "0"+tmp['Monthly Reporting Period'].apply(str), 
                         tmp['Monthly Reporting Period'].apply(str))
tmp['ASOF_DATE'] = pd.to_datetime(tmp['ASOF_DATE'], format='%d%m%Y', errors='coerce')
tmp = tmp.set_index(['ASOF_DATE'])

tmp = tmp.apply(pd.to_numeric, errors='coerce')
tmp = tmp.sort_values(by=['ASOF_DATE'], ascending=[True])
tmp = tmp.reset_index()


In [10]:
#calculate CPR

cpr_df = tmp.copy()

cpr_df[['Previous UPB', 'Previous Interest Rate']] = cpr_df.groupby(['Reference Pool ID', 'Loan Identifier'])[['Current Actual UPB', 'Current Interest Rate']].shift()
cpr_df['Previous UPB'] = cpr_df['Previous UPB'].fillna(cpr_df['Current Actual UPB'])
cpr_df['Previous Interest Rate'] = cpr_df['Previous Interest Rate'].fillna(cpr_df['Current Interest Rate'])

cpr_df['Unscheduled Pmt Pct'] = (cpr_df['Unscheduled Principal Current'] / cpr_df['Previous UPB'].fillna(0)).fillna(0)
cpr_df['ARM'] = (cpr_df['Unscheduled Principal Current'] * 12).fillna(0)
cpr_df['CPR'] = cpr_df['ARM'] / cpr_df['Previous UPB'].fillna(0) * 100


In [11]:
#bring in rate incentive
rt_df = cpr_df.copy()
rt_df = rt_df.merge(me_tsy_df[['ME_DATE', 'DGS10']], how='left', left_on='ASOF_DATE', right_on='ME_DATE')
rt_df['Rate Incentive'] = ((rt_df['DGS10']) - rt_df['Previous Interest Rate'] )
rt_df = rt_df.dropna(subset=['Rate Incentive'])


In [12]:
#calculate if ever delinquent
dq_df = rt_df.copy()
dq_df['Current Loan Delinquency Status'] = pd.to_numeric(dq_df['Current Loan Delinquency Status'])
dq_df = dq_df.loc[dq_df['Current Loan Delinquency Status'] > 0]
edq_df = rt_df.copy()
edq_df['EVER_DQ'] = edq_df['Loan Identifier'].isin(dq_df['Loan Identifier'].unique())


In [13]:
# inc_df = edq_df.copy()
# inc_df['Inc'] = pd.cut(x = inc_df['Rate Incentive'], bins = 50, labels = False, include_lowest = True)
# inc_df['Uns'] = pd.cut(x = inc_df['Unscheduled Pmt Pct'], bins = 50, labels = False, include_lowest = True)
# inc_df.loc[inc_df['Inc'] == 40]



In [34]:
def st_grp(group):
    tot_bal = group['Current Actual UPB'].sum()
    print(group.head(10))

count_bins = 10
#filter on: 
# 'Number of Borrowers'
# 'Zip Code Short'
# 'Original Interest Rate'
    
    
bucket_df = edq_df.copy()    
bucket_df = bucket_df.loc[bucket_df['ASOF_DATE'] == min(bucket_df['ASOF_DATE'])]
tot_bal = bucket_df['Original UPB'].sum()
bucket_df['Bal Pct'] = bucket_df['Original UPB'] / tot_bal
bucket_df['Rate Bin'] = pd.cut(x = bucket_df['Original Interest Rate'], bins = count_bins, labels = False, include_lowest = True)
bucket_df['LTV Bin'] = pd.cut(x = bucket_df['Original Combined Loan to Value Ratio (CLTV)'], bins = count_bins, labels = False, include_lowest = True)
bucket_df['FICO Bin'] = pd.cut(x = bucket_df['Borrower Credit Score At Issuance'], bins = count_bins, labels = False, include_lowest = True)
bucket_df['DTI Bin'] = pd.cut(x = bucket_df['Debt-To-Income (DTI)'], bins = count_bins, labels = False, include_lowest = True)

col_df = bucket_df[['EVER_DQ', 'Original UPB', 'Bal Pct', 'Rate Bin', 'LTV Bin', 'FICO Bin', 'DTI Bin']]
col_df

,EVER_DQ,Original UPB,Bal Pct,Rate Bin,LTV Bin,FICO Bin,DTI Bin
0,False,174000.000,0.000,2,8,8.000,3
1,False,57000.000,0.000,5,8,7.000,7
2,False,94000.000,0.000,4,5,9.000,3
3,False,88000.000,0.000,3,9,6.000,8
4,True,77000.000,0.000,5,9,6.000,6
...,...,...,...,...,...,...,...
125568,True,248000.000,0.000,3,5,7.000,7
125569,False,207000.000,0.000,3,5,7.000,9
125570,False,195000.000,0.000,2,8,8.000,6
125571,False,266000.000,0.000,4,1,8.000,4


In [31]:
x_col = "LTV Bin"

grp_df = col_df.groupby([x_col, 'EVER_DQ'])['Bal Pct'].sum().reset_index(drop=False)
# grp_df['Bal Pct'].sum()
# grp_df
alt.Chart(grp_df).mark_bar().encode(
    x=x_col,
    y='Bal Pct',
    color='EVER_DQ'
)


ValueError: Grouper for 'LTV Bin' not 1-dimensional

## Hypothesis 1: Viewing distribution shapes of major credit attributes

In [11]:
alt.Chart(c_df_orig.sample(1000)).mark_bar().\
    encode(x = 'Borrower Credit Score At Issuance', y = 'count()')

alt.Chart(...)

In [12]:
alt.Chart(c_df_orig.sample(1000)).mark_bar().\
    encode(x = 'Debt-To-Income (DTI)', y = 'count()')

alt.Chart(...)

In [13]:
alt.Chart(c_df_orig.sample(1000)).mark_bar().\
    encode(x = 'Original Loan to Value Ratio (LTV)', y = 'count()')

alt.Chart(...)